In [7]:
from sklearn.neighbors import LocalOutlierFactor
import pandas as pd
import numpy as np
from sklearn import preprocessing

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

sqlCtx = SQLContext(sc)

df = sqlCtx.read.load('file:///home/juanjo/Desktop/MachineLearning/data/interim/T2_Labels.csv',
                     format='com.databricks.spark.csv', header='true', inferSchema='true')



In [14]:
drop_list = ['UserID', 'UUID', 'Version', 'TimeStemp']
df = df.select([column for column in df.columns if column not in drop_list])

In [31]:
percentile = 1
n_outliers = len(X_scores[X_scores < np.percentile(X_scores, percentile)])
lowest_indices = np.argsort(-X_scores, axis=0)[-1:-1-n_outliers:-1]
outliers = df.toPandas().iloc[lowest_indices]

outliers = sqlCtx.createDataFrame(outliers)

In [36]:
attacks = df.filter(df.labels == 1)
outliers = outliers.filter(outliers.labels != 1)

outliers = outliers.union(attacks)

outliers.write.csv("/home/juanjo/Desktop/MachineLearning/data/processed/T2_sample_per1.csv")

In [39]:
####### OUTLIERS + INLIERS : PERCENTILE 1
percentile = 0.5
n_outliers = len(X_scores[X_scores < np.percentile(X_scores, percentile)])
lowest_indices = np.argsort(-X_scores, axis=0)[-1:-1-n_outliers:-1]
outliers = df.toPandas().iloc[lowest_indices]
outliers = sqlCtx.createDataFrame(outliers)

In [41]:
attacks = df.filter(df.labels == 1)
outliers = outliers.filter(outliers.labels != 1)
outliers = outliers.union(attacks)

In [49]:
inliers = df.sample(False, n_outliers/df.count(), 42)
inliers = inliers[(inliers['labels'] == 0)]
inliers = inliers.filter(inliers.labels == 0)

In [51]:
merged = outliers.union(inliers)
merged.write.csv("/home/juanjo/Desktop/MachineLearning/data/processed/T2_sample_per1_inliers.csv")